In [1]:
%pip install tensorboard --quiet
%pip install seaborn --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('/Users/steeeve/Documents/csiss/streetview_crop_classification/cropClassification')
sys.path.append('/Users/steeeve/Documents/csiss/streetview_crop_classification/cropClassification/model')
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
import pandas as pd
from torch.utils.data import DataLoader

from unet import originalUNet
from compiler import ModelCompiler
from dataloader import RoadsideCropImageDataset
from loss import BalancedCrossEntropyLoss

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU with CUDA
    print("Using CUDA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Apple M1/M2 GPU with MPS (Metal Performance Shaders)
    print("Using MPS")
else:
    device = torch.device("cpu")  # Fall back to CPU
    print("Using CPU")

Using MPS


### Configuration

In [4]:
config = {
    "model": {
        "type": "originalUNet",  # Example model type, replace with your actual model class name
        "params": {
            "in_channels": 9,  # Since we are using 9-channel input images
            "out_channels": 3   # Number of output classes for segmentation
        }
    },
    "training": {
        "epochs": 50,
        "batch_size": 32,
        "learning_rate": 0.01,
        "optimizer": {
            "type": "Adam",
            "params": {
                "lr": 0.01
            }
        },
        "scheduler": {
            "type": "StepLR",
            "params": {
                "step_size": 10,
                "gamma": 0.5
            }
        },
        "criterion": BalancedCrossEntropyLoss(),  # or you can use your custom loss like BalancedCrossEntropyLoss
        "resume" : False,
        "resume_epoch" : None,
    },
    "validation": {
        "epochs": 50,
        "batch_size": 16
    },
    "dataset": {
        "train_csv": "/Users/steeeve/Documents/csiss/data/masked_data_csiss/training/train_chipping_csv.csv",  # Path to the training DataFrame (includes npy file paths)
        "val_csv": "/Users/steeeve/Documents/csiss/data/masked_data_csiss/validation/validation_chipping_csv.csv", # Path to the validation DataFrame (includes npy file paths)
        "train_root_path": "/Users/steeeve/Documents/csiss/data/masked_data_csiss/training",
        "val_root_path": "/Users/steeeve/Documents/csiss/data/masked_data_csiss/validation",
        "image_column": "img_chip_path",             # Column containing the image paths (npy files)
        "mask_column": "lbl_chip_path",              # Column containing the mask paths
        "train_mean": [93.8785585, 111.81092494, 76.94555781, 113.58929434, 206.93557473, 28.98472963,
                       46.53684269, 113.74437327, 116.23856585],  # Mean values for training set normalization
        "train_std": [53.24595916, 46.34658429, 45.91157286, 47.69937365, 82.32605363, 48.02506071,
                       22.46416468, 52.31732116, 47.89290138],   # Std values for training set normalization
        "val_mean": [88.62211239, 111.27628711, 75.19030815, 111.99622799, 223.21125121, 25.98495476, 
                     49.64829283, 115.91463906, 114.01573621],    # Mean values for validation set normalization
        "val_std": [53.97481266, 46.47043658, 45.64034871, 48.20284577, 59.89659002, 41.7467965,
                     20.82148233, 54.41768437, 47.68361442],     # Std values for validation set normalization
    },
    "evaluation": {
        "filename": "csiss_street_view_crop_classification.csv",  # Evaluation metrics to be used
        "class_mapping": {
            0: "Background",
            1: "Maize",
            2: "Soybean"
        }
    }
}


### Loading Datasets

In [5]:
train_df = pd.read_csv(config['dataset']['train_csv'])
val_df = pd.read_csv(config['dataset']['val_csv'])

In [6]:
train_dataset = RoadsideCropImageDataset(
    dataframe=train_df,
    root_dir=config['dataset']['train_root_path'],  # Root directory where images are stored
    usage='train',  # Indicates training dataset
    mean=config['dataset']['train_mean'],
    std=config['dataset']['train_std']
)

val_dataset = RoadsideCropImageDataset(
    dataframe=val_df,
    root_dir=config['dataset']['val_root_path'],  # Root directory where images are stored
    usage='val',  # Indicates validation dataset
    mean=config['dataset']['val_mean'],
    std=config['dataset']['val_std']
)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=config['training']['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['training']['batch_size'], shuffle=False)

### Construct Model and Training

In [8]:
model = originalUNet(n_channels=config['model']['params']['in_channels'],
                     n_classes=config['model']['params']['out_channels'])
model_comp = ModelCompiler(model=model,
                           params_init=None)

---------- MPS available ----------
Total number of trainable parameters: 13.4M
---------- originalUNet model compiled successfully ----------


In [9]:
model_comp.fit(
    trainDataset=train_loader,
    valDataset=val_loader,
    epochs=config['training']['epochs'],
    optimizer_name=config['training']['optimizer']['type'],
    lr_init=config['training']['learning_rate'],
    lr_policy='steplr',  # Use 'steplr' as specified in config
    criterion=config['training']['criterion'],
    log=True,  # Enable logging for TensorBoard
    **config['training']['scheduler']['params'])

-------------------------- Start training --------------------------
[1/50]
Batch 0/115: Loss 1.0951
Batch 10/115: Loss 1.0463
Batch 20/115: Loss 0.9203
Batch 30/115: Loss 0.8679
Batch 40/115: Loss 0.8778
Batch 50/115: Loss 0.8237
Batch 60/115: Loss 0.7205
Batch 70/115: Loss 0.6664
Batch 80/115: Loss 0.7753
Batch 90/115: Loss 0.8091
Batch 100/115: Loss 0.7687
Batch 110/115: Loss 0.6936
Epoch Training Loss: 0.8227
Current Learning Rate: 0.000005
Batch 0/30: Validation Loss 1.0287
Batch 10/30: Validation Loss 0.6222
Batch 20/30: Validation Loss 0.9985
Validation Loss: 0.7488
[2/50]
Batch 0/115: Loss 0.7033
Batch 10/115: Loss 0.6355
Batch 20/115: Loss 0.7742
Batch 30/115: Loss 0.7315
Batch 40/115: Loss 0.6196
Batch 50/115: Loss 0.7529
Batch 60/115: Loss 0.8065
Batch 70/115: Loss 0.7661
Batch 80/115: Loss 0.6818
Batch 90/115: Loss 0.7775
Batch 100/115: Loss nan
Batch 110/115: Loss nan
Epoch Training Loss: nan
Current Learning Rate: 0.000000
Batch 0/30: Validation Loss nan


KeyboardInterrupt: 